# LangChain Few-Shot Message Prompt Templates

This notebook demonstrates how to use **few-shot message prompt templates**
in LangChain to guide model behavior using example conversations.

The focus is on:
- Defining reusable human and AI message templates
- Providing multiple example interactions (few-shot learning)
- Composing few-shot prompts with new user inputs
- Executing structured prompts using `ChatOpenAI`

Few-shot prompting helps models infer tone, style, and expected behavior
without explicit instructions.


In [ ]:
import getpass
import os
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ( HumanMessagePromptTemplate, ChatPromptTemplate, FewShotChatMessagePromptTemplate, AIMessagePromptTemplate)


In [ ]:
if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

## Model Initialization

A chat-based OpenAI model is initialized with deterministic settings
to ensure consistent behavior across runs.


In [ ]:
chat = ChatOpenAI(
    model="gpt-5-nano", 
    temperature=0, 
    model_kwargs= {"text":{"verbosity": 'low'},"reasoning":{"effort": "medium"}},
    
    ) 

## Defining Human and AI Message Templates

Message templates define reusable prompt structures with placeholders.

- The **human template** represents the user’s request
- The **AI template** represents an example assistant response

These templates are later combined to form few-shot examples.


In [ ]:
TEMPLATE_H= "I've recently adopted a {pet}. Could you suggest some {pet} names?"
TEMPLATE_AI= "{response}"

message_template_h =  HumanMessagePromptTemplate.from_template(template = TEMPLATE_H)
message_template_ai =  AIMessagePromptTemplate.from_template(template = TEMPLATE_AI)

## Creating a Few-Shot Example Template

A `ChatPromptTemplate` is used to define the structure
of each example interaction.

Each example consists of:
- A human message
- A corresponding AI response


In [ ]:
example_template =  ChatPromptTemplate.from_messages([message_template_h, message_template_ai])

## Defining Few-Shot Examples

Multiple example interactions are provided to demonstrate
the desired response style.

These examples help the model infer tone, humor, and structure
without additional instructions.


In [ ]:
examples = [
            {"pet": "dog", "response": '''Oh, absolutely. Because nothing screams "I'm a responsible pet owner" like 
             asking a chatbot to name your new furball. How about "Bark Twain" (if it's a literary hound)?''' },
            {"pet": "cat",  "response": ''' Oh, absolutely. Because nothing screams "I'm a unique and creative individual" like asking a 
             chatbot to name your cat, How about "Furry McFurFace" , "Sir Meowsalot", or "Catastrophe" ?'''},
             {"pet": "parrot", "response": '''Oh, absolutely. Because nothing screams "I'm a quirky and fun pet owner" like asking a chatbot to name your parrot.
             How about "Polly Tician" (for the politically inclined bird) or "Squawk Ness Monster" ?'''},
            ]

## Constructing the Few-Shot Prompt

The examples are wrapped using `FewShotChatMessagePromptTemplate`,
which automatically expands them into a sequence of messages
at runtime.


In [ ]:
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_template,
    examples=examples,
)


## Composing the Final Chat Prompt

The few-shot examples are combined with a new human message,
allowing the model to respond to unseen inputs
while following the demonstrated style.


In [ ]:
chat_template =  ChatPromptTemplate.from_messages([few_shot_prompt, message_template_h])

In [ ]:
chat_value = chat_template.invoke({"pet": "rabbit"})

## Inspecting the Expanded Prompt

The expanded messages can be inspected to verify
how examples and user input are assembled
before being sent to the model.


In [ ]:
chat_value

In [ ]:
for i in chat_value.messages:
    print(f'{i.type}: {i.content}\n')

## Executing the Few-Shot Prompt

The structured few-shot prompt is passed to the chat model,
which generates a response consistent with the provided examples.


In [ ]:
response = chat.invoke(chat_value)

In [ ]:
print(response.text)

## Summary

This notebook demonstrated:

- Creating few-shot message prompt templates in LangChain  
- Defining reusable human and AI message structures  
- Providing multiple example interactions to guide model behavior  
- Composing and executing few-shot prompts using `ChatOpenAI`  

Few-shot message templates are a powerful abstraction for
controlling tone and behavior without hard-coded rules.
